In [31]:
# !pip install -q ctransformers
# !pip install -q transformers
# !pip install -q langchain
# !pip install -q langchain-community
# !pip install -q torch
# !pip install -q pypdf
# !pip install -q sentence-transformers
# !pip install -q gpt4all
# !pip install -q faiss-cpu
# !pip install -q openai
# !pip install -q bitsandbytes accelerate
# !pip install -q -U bitsandbytes accelerate gpt4all

In [32]:
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.llms import CTransformers
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# **Embedding and Store vector DB**

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings


In [34]:
# !ls -a /content

In [35]:
# from google.colab import drive
# drive.mount('/content/drive')

In [36]:
# Khai bao bien
data_path = "E:/LLM/RagMiAI/testAn/data"
vector_db_path = "E:/LLM/RagMiAI/testAn/vectorstores"

In [37]:
loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

texts = splitter.split_documents(documents)

In [38]:
print(len(documents))
print(len(texts))

4
28


In [39]:
len(texts)

28

In [40]:
texts[10]

Document(metadata={'source': 'E:\\LLM\\RagMiAI\\testAn\\data\\Chinh_tri.pdf', 'page': 1}, page_content='quốc lần thứ XIII đã đ ề ra, tổ chức thành công Đ ại hội XIV, đưa đ ất nước vững bư ớc phát tri ển trong giai \nđoạn mới. Ông Tô Lâm sinh ngày 10/7/1957; quê xã Nghĩa Tr ụ, huy ện Văn Giang, t ỉnh Hưng Yên; trình đ ộ \nchuyên môn Giáo sư, Ti ến sĩ Lu ật học.')

In [41]:
embedding_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

c:\Users\Snape\anaconda3\envs\torch\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
db = FAISS.from_documents(texts, embeddings)
db.save_local(vector_db_path)

# **Retriver**

In [43]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline


In [44]:
def load_llm(model_file):
    llm = CTransformers(
        model=model_file,
        model_type="llama",
        max_new_tokens=1024,
        temperature=0.01
    )
    return llm


In [45]:
# Cau hinh
model_file = "E:/LLM/RagMiAI/models/vinallama-7b-chat_q5_0.gguf"
vector_db_path = "E:/LLM/RagMiAI/testAn/vectorstores"

llm = load_llm(model_file)

In [46]:
retriever = db.as_retriever(search_kwargs={"k": 3}, max_tokens_limit=520)
docs = retriever.get_relevant_documents("Thượng tướng Lương Tam Quang")

In [47]:
docs[0]

Document(metadata={'source': 'E:\\LLM\\RagMiAI\\testAn\\data\\Chinh_tri.pdf', 'page': 0}, page_content='Thượng tư ớng Lương Tam Quang, Ủy viên Trung ương Đ ảng, B ộ trưởng B ộ Công an đư ợc Ban Ch ấp \nhành Trung ương Đ ảng b ầu bổ sung vào B ộ Chính tr ị khóa 13.  \nThông tin t ừ Văn phòng Trung ương Đ ảng cho bi ết, chi ều 16/8, Ban Ch ấp hành Trung ương Đ ảng khóa 13 \nhọp về công tác cán b ộ. \nCụ thể, Ban Ch ấp hành Trung ương Đ ảng đã b ầu bổ sung Thư ợng tư ớng Lương Tam Quang , Ủy viên \nTrung ương Đ ảng, B ộ trưởng B ộ Công an vào B ộ Chính tr ị khóa 13.')

In [48]:
docs[1]

Document(metadata={'source': 'E:\\LLM\\RagMiAI\\testAn\\data\\Chinh_tri.pdf', 'page': 0}, page_content='Thượng tư ớng Lương Tam Quang sinh năm 1965; quê xã Hi ệp Cư ờng, huy ện Kim Đ ộng, t ỉnh Hưng Yên; \ntrình đ ộ Cử nhân Lu ật, An ninh.  \nÔng là Ủy viên Trung ương Đ ảng khóa 13, Bí thư Đ ảng ủy Công an Trung ương, B ộ trưởng B ộ Công an, Ủy \nviên H ội đồng qu ốc phòng và an ninh (t ừ tháng 6/2024).   \nTrong quá trình công tác, ông Lương Tam Quang t ừng đ ảm nh ận nhi ều ch ức vụ trong lĩnh v ực an ninh, \nlàm tr ợ lý Th ứ trưởng B ộ Công an.')

In [49]:
#Tao Prompt
template = """<|im_start|>system\nSử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời\n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""


prompt = PromptTemplate(template=template)

# **Make a Chain**

In [50]:
llm_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={'prompt': prompt}
)


In [51]:
question = "Thượng tướng Lương Tam Quang là ai?"
response = llm_chain.invoke(question)
print(response)

Number of tokens (553) exceeded maximum context length (512).
Number of tokens (554) exceeded maximum context length (512).
Number of tokens (555) exceeded maximum context length (512).
Number of tokens (556) exceeded maximum context length (512).
Number of tokens (557) exceeded maximum context length (512).
Number of tokens (558) exceeded maximum context length (512).
Number of tokens (559) exceeded maximum context length (512).
Number of tokens (560) exceeded maximum context length (512).
Number of tokens (561) exceeded maximum context length (512).
Number of tokens (562) exceeded maximum context length (512).
Number of tokens (563) exceeded maximum context length (512).
Number of tokens (564) exceeded maximum context length (512).
Number of tokens (565) exceeded maximum context length (512).
Number of tokens (566) exceeded maximum context length (512).
Number of tokens (567) exceeded maximum context length (512).
Number of tokens (568) exceeded maximum context length (512).
Number o

{'query': 'Thượng tướng Lương Tam Quang là ai?', 'result': '\n    .  \nTrident of the 13thunt.\nBan ch \n\n\n\nNgười phát minh vầnt_info\nChủ tịch thứ nhất của Uỷ viên Hội đồng hie\n\n  \n  \n\n\n\n\nBan chủanhant\n\nQuốc hội nghị, và được chỉ huy hiệu trưởng B \n\n\n\nMã|]gner\n\n\n'}
